# ===============================================
# Video Stutter Detection Notebook
# このノートブックは簡易実験用です
# 実行環境（カーネル）はTestProg_video（Python 3.8.19）で行ってください
# ===============================================

In [ ]:
import cv2

def list_camera_modes(device_index=0):
    cap = cv2.VideoCapture(device_index, cv2.CAP_MSMF)  # Media Foundation 経由
    if not cap.isOpened():
        print("カメラが開けませんでした")
        return

    # 確認したい解像度とFPSの候補
    resolutions = [(1920,1080), (1280,720), (640,480)]
    fps_list = [15, 30, 60, 120]

    for w, h in resolutions:
        for fps in fps_list:
            cap.set(cv2.CAP_PROP_FRAME_WIDTH, w)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT, h)
            cap.set(cv2.CAP_PROP_FPS, fps)

            actual_w  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            actual_h  = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            actual_fps = cap.get(cv2.CAP_PROP_FPS)

            print(f"要求: {w}x{h} @ {fps}fps → 実際: {int(actual_w)}x{int(actual_h)} @ {int(actual_fps)}fps")

    cap.release()

if __name__ == "__main__":
    list_camera_modes(0)  # 内蔵カメラを確認 (USBカメラなら番号を変える)

↓動作しない

In [ ]:
import os
import shutil
import cv2
import numpy as np
import threading
import queue
import time

# ====================================================
# カクつき検知ワーカー（最初のフレームのみ保存）
# ====================================================
def stutter_worker_first_only(frame_queue, output_folder, fps=60, threshold=100, min_time_diff=0.1):
    min_frame_diff = int(min_time_diff * fps)
    prev_gray = None
    temp_stutter_indices = []
    stutter_frames = []
    last_diff_time = time.time()

    os.makedirs(output_folder, exist_ok=True)

    while True:
        item = frame_queue.get()
        if item is None:  # 終了シグナル
            break

        idx, frame = item
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            non_zero_count = np.count_nonzero(diff)

            if non_zero_count <= threshold:
                temp_stutter_indices.append((idx, frame))
            else:
                if len(temp_stutter_indices) >= min_frame_diff:
                    # 最初のフレームのみ保存
                    first_idx, first_frame = temp_stutter_indices[0]
                    stutter_frames.append(first_idx)
                    cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)
                temp_stutter_indices = []
                last_diff_time = time.time()  # 差分があったので更新

        prev_gray = gray
        frame_queue.task_done()

        # 差分無しで5分以上なら停止
        if time.time() - last_diff_time > stop_no_diff_sec:
            print("5分以上変化なしのためキャプチャ停止")
            break

    # 残りも処理
    if len(temp_stutter_indices) >= min_frame_diff:
        first_idx, first_frame = temp_stutter_indices[0]
        stutter_frames.append(first_idx)
        cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)

    frame_queue.task_done()
    print(f"{len(stutter_frames)} フレームがカクつきとして検出され、最初のフレームのみ保存されました。")


# ====================================================
# キャプチャ＆逐次カクつき検知
# ====================================================
def start_capture_and_detect(device_number, max_frames=None, width=720, height=480,
                             capture_fps=60, display_fps=10, threshold=100, 
                             min_time_diff=0.1, max_temp_frames=36000, stop_no_diff_sec=300):
    # 一時フォルダ作成
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    temp_folder = os.path.join(desktop, "temp_frames")
    output_folder = os.path.join(desktop, "stutter_frames")

    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    os.makedirs(temp_folder, exist_ok=True)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    print(f"{temp_folder} と {output_folder} を作成しました。")

    # キャプチャ初期化
    cap = cv2.VideoCapture(device_number, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Error: キャプチャデバイスを開けませんでした")
        return 0, [], 0

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, capture_fps)

    # カクつき検知用キュー＆スレッド
    frame_queue = queue.Queue(maxsize=10)
    worker = threading.Thread(target=stutter_worker_first_only,
                              args=(frame_queue, output_folder, capture_fps, threshold, min_time_diff),
                              daemon=True)
    worker.start()

    frame_count = 0
    stutter_frames = []
    prev_frame = None
    last_diff_time = time.time()
    display_interval = 1.0 / display_fps
    prev_display_time = time.time()
    start_time = time.time()

    print("映像キャプチャ開始。'q'キーで終了できます。")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # フレームを一時保存
        frame_file = os.path.join(temp_folder, f"frame_{frame_count:05d}.png")
        cv2.imwrite(frame_file, frame)

        # 検知キューに追加
        frame_queue.put((frame_count, frame.copy()))

        # 一時フォルダが36,000枚を超えたらクリア
        if frame_count % max_temp_frames == 0:
            shutil.rmtree(temp_folder)
            os.makedirs(temp_folder, exist_ok=True)
            print(f"{max_temp_frames} 枚保存後、一時フォルダをクリアしました")

        # 表示更新
        curr_time = time.time()
        if curr_time - prev_display_time >= display_interval:
            fps_display = 1.0 / (curr_time - prev_display_time)
            prev_display_time = curr_time
            display_frame = frame.copy()
            cv2.putText(display_frame, f"Display FPS: {fps_display:.2f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Preview (FPS limited)", display_frame)

        frame_count += 1

        # 終了条件
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("ユーザーによる終了")
            break
        if max_frames is not None and frame_count >= max_frames:
            print(f"最大フレーム数 {max_frames} に到達")
            break

    # キャプチャ終了
    cap.release()
    cv2.destroyAllWindows()

    # 検知スレッド終了
    frame_queue.put(None)
    frame_queue.join()

    total_time = time.time() - start_time
    actual_fps = frame_count / total_time if total_time > 0 else 0

    print(f"{frame_count} フレームをキャプチャしました（実際のFPS: {actual_fps:.2f}）")
    return frame_count, output_folder, actual_fps


# ====================================================
# 実行例
# ====================================================
if __name__ == "__main__":
    start_capture_and_detect(device_number=0, max_frames=None, capture_fps=60, display_fps=10,
                             threshold=100, min_time_diff=0.1)


↓動作する
確認項目
１．5分以上の差分なしは停止として処理
２．36000枚（10分）の画像がたまったら画像データを破棄

In [ ]:
import os
import shutil
import cv2
import numpy as np
import threading
import queue
import time

# ====================================================
# カクつき検知ワーカー（最初のフレームのみ保存）
# ====================================================
def stutter_worker_first_only(frame_queue, output_folder, fps=60, threshold=100, min_time_diff=0.1, stop_no_diff_sec=300):
    min_frame_diff = int(min_time_diff * fps)
    prev_gray = None
    temp_stutter_indices = []
    stutter_frames = []
    last_diff_time = time.time()  # 追加

    os.makedirs(output_folder, exist_ok=True)

    while True:
        item = frame_queue.get()
        if item is None:  # 終了シグナル
            break

        idx, frame = item
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            non_zero_count = np.count_nonzero(diff)

            if non_zero_count <= threshold:
                temp_stutter_indices.append((idx, frame))
            else:
                if len(temp_stutter_indices) >= min_frame_diff:
                    first_idx, first_frame = temp_stutter_indices[0]
                    stutter_frames.append(first_idx)
                    cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)
                temp_stutter_indices = []
                last_diff_time = time.time()  # 差分があったので更新

        prev_gray = gray
        frame_queue.task_done()

        # 差分無しで指定秒以上なら停止
        if time.time() - last_diff_time > stop_no_diff_sec:
            print("5分以上変化なしのためキャプチャ停止")
            break

    # 残りも処理
    if len(temp_stutter_indices) >= min_frame_diff:
        first_idx, first_frame = temp_stutter_indices[0]
        stutter_frames.append(first_idx)
        cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)

    frame_queue.task_done()
    print(f"{len(stutter_frames)} フレームがカクつきとして検出され、最初のフレームのみ保存されました。")


# ====================================================
# キャプチャ＆逐次カクつき検知
# ====================================================
def start_capture_and_detect(device_number, max_frames=None, width=720, height=480,
                             capture_fps=60, display_fps=10, threshold=100, 
                             min_time_diff=0.1, max_temp_frames=36000, stop_no_diff_sec=300):
    # 一時フォルダ作成
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    temp_folder = os.path.join(desktop, "temp_frames")
    output_folder = os.path.join(desktop, "stutter_frames")

    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    os.makedirs(temp_folder, exist_ok=True)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    print(f"{temp_folder} と {output_folder} を作成しました。")

    # キャプチャ初期化
    cap = cv2.VideoCapture(device_number, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Error: キャプチャデバイスを開けませんでした")
        return 0, output_folder, 0

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, capture_fps)

    # カクつき検知用キュー＆スレッド
    frame_queue = queue.Queue(maxsize=10)
    worker = threading.Thread(target=stutter_worker_first_only,
                              args=(frame_queue, output_folder, capture_fps, threshold, min_time_diff, stop_no_diff_sec),
                              daemon=True)
    worker.start()

    frame_count = 0
    display_interval = 1.0 / display_fps
    prev_display_time = time.time()
    start_time = time.time()  # 追加

    print("映像キャプチャ開始。'q'キーで終了できます。")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # フレームを一時保存
        frame_file = os.path.join(temp_folder, f"frame_{frame_count:05d}.png")
        cv2.imwrite(frame_file, frame)

        # 検知キューに追加
        frame_queue.put((frame_count, frame.copy()))

        # 一時フォルダが36,000枚を超えたらクリア
        if frame_count != 0 and frame_count % max_temp_frames == 0:
            shutil.rmtree(temp_folder)
            os.makedirs(temp_folder, exist_ok=True)
            print(f"{max_temp_frames} 枚保存後、一時フォルダをクリアしました")

        # 表示更新
        curr_time = time.time()
        if curr_time - prev_display_time >= display_interval:
            fps_display = 1.0 / (curr_time - prev_display_time)
            prev_display_time = curr_time
            display_frame = frame.copy()
            cv2.putText(display_frame, f"Display FPS: {fps_display:.2f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Preview (FPS limited)", display_frame)

        frame_count += 1

        # 終了条件
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("ユーザーによる終了")
            break
        if max_frames is not None and frame_count >= max_frames:
            print(f"最大フレーム数 {max_frames} に到達")
            break

    # キャプチャ終了
    cap.release()
    cv2.destroyAllWindows()

    # 検知スレッド終了
    frame_queue.put(None)
    frame_queue.join()

    total_time = time.time() - start_time
    actual_fps = frame_count / total_time if total_time > 0 else 0

    print(f"{frame_count} フレームをキャプチャしました（実際のFPS: {actual_fps:.2f}）")
    return frame_count, output_folder, actual_fps


# ====================================================
# 実行例
# ====================================================
if __name__ == "__main__":
    start_capture_and_detect(device_number=0, max_frames=None, capture_fps=60, display_fps=10,
                             threshold=100, min_time_diff=0.1)


↓動作する※動作するやつのメモリ消費改善版

In [ ]:
import os
import shutil
import cv2
import numpy as np
import threading
import queue
import time
import gc

# ====================================================
# カクつき検知ワーカー（最初のフレームのみ保存）
# ====================================================
def stutter_worker_first_only(frame_queue, output_folder, fps=60, threshold=100, min_time_diff=0.1, stop_no_diff_sec=300):
    min_frame_diff = int(min_time_diff * fps)
    prev_gray = None
    temp_stutter_indices = []
    stutter_frames = []
    last_diff_time = time.time()

    os.makedirs(output_folder, exist_ok=True)

    while True:
        item = frame_queue.get()
        if item is None:
            break

        idx, frame = item
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        diff = None  # 初期化
        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            non_zero_count = np.count_nonzero(diff)

            if non_zero_count <= threshold:
                temp_stutter_indices.append((idx, frame))
            else:
                if len(temp_stutter_indices) >= min_frame_diff:
                    first_idx, first_frame = temp_stutter_indices[0]
                    stutter_frames.append(first_idx)
                    cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)
                temp_stutter_indices = []
                last_diff_time = time.time()

        prev_gray = gray

        # メモリ解放
        if diff is not None:
            del diff
        del gray, frame
        import gc; gc.collect()

        frame_queue.task_done()

        # 差分無しで指定秒以上なら停止
        if time.time() - last_diff_time > stop_no_diff_sec:
            print("5分以上変化なしのためキャプチャ停止")
            cv2.destroyAllWindows()  # プレビューウィンドウを閉じる
            break

        # キューに追加（非ブロック）
        try:
            frame_queue.put_nowait((frame_count, frame))
        except queue.Full:
            # キューが満杯ならフレームを破棄
            del frame
            import gc; gc.collect()
            continue

        frame_count += 1        

    # 残り処理
    if len(temp_stutter_indices) >= min_frame_diff:
        first_idx, first_frame = temp_stutter_indices[0]
        stutter_frames.append(first_idx)
        cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)

    print(f"{len(stutter_frames)} フレームがカクつきとして検出され、最初のフレームのみ保存されました。")



# ====================================================
# キャプチャ＆逐次カクつき検知（メモリ効率版）
# ====================================================
def start_capture_and_detect(device_number, width=720, height=480,
                             capture_fps=60, display_fps=10, threshold=100, 
                             min_time_diff=0.1, max_temp_frames=36000, stop_no_diff_sec=300):
    # 一時フォルダ作成
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    temp_folder = os.path.join(desktop, "temp_frames")
    output_folder = os.path.join(desktop, "stutter_frames")

    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    os.makedirs(temp_folder, exist_ok=True)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    print(f"{temp_folder} と {output_folder} を作成しました。")

    # キャプチャ初期化
    cap = cv2.VideoCapture(device_number, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Error: キャプチャデバイスを開けませんでした")
        return 0, output_folder, 0

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, capture_fps)

    # カクつき検知用キュー＆スレッド
    frame_queue = queue.Queue(maxsize=5)  # 小さめにしてメモリ増加防止
    worker = threading.Thread(target=stutter_worker_first_only,
                              args=(frame_queue, output_folder, capture_fps, threshold, min_time_diff, stop_no_diff_sec),
                              daemon=True)
    worker.start()

    frame_count = 0
    display_interval = 1.0 / display_fps
    prev_display_time = time.time()
    start_time = time.time()

    print("映像キャプチャ開始。'q'キーで終了できます。")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # フレームを一時保存
        frame_file = os.path.join(temp_folder, f"frame_{frame_count:05d}.png")
        cv2.imwrite(frame_file, frame)

        # 検知キューに追加
        frame_queue.put((frame_count, frame))  # copy しない

        # 一時フォルダが36,000枚を超えたらクリア
        if frame_count != 0 and frame_count % max_temp_frames == 0:
            shutil.rmtree(temp_folder)
            os.makedirs(temp_folder, exist_ok=True)
            print(f"{max_temp_frames} 枚保存後、一時フォルダをクリアしました")

        # 表示更新
        curr_time = time.time()
        if curr_time - prev_display_time >= display_interval:
            fps_display = 1.0 / (curr_time - prev_display_time)
            prev_display_time = curr_time
            display_frame = frame.copy()
            cv2.putText(display_frame, f"Display FPS: {fps_display:.2f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Preview (FPS limited)", display_frame)

        frame_count += 1

        # 終了条件
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("ユーザーによる終了")
            break

        # 手動停止なしなら継続（差分5分なしでワーカーが停止）

        # メモリ解放
        del frame
        gc.collect()

    # キャプチャ終了
    cap.release()
    cv2.destroyAllWindows()

    # 検知スレッド終了
    frame_queue.put(None)
    frame_queue.join()

    total_time = time.time() - start_time
    actual_fps = frame_count / total_time if total_time > 0 else 0

    print(f"{frame_count} フレームをキャプチャしました（実際のFPS: {actual_fps:.2f}）")
    return frame_count, output_folder, actual_fps


# ====================================================
# 実行例
# ====================================================
if __name__ == "__main__":
    start_capture_and_detect(device_number=0, capture_fps=60, display_fps=10,
                             threshold=100, min_time_diff=0.1)


In [ ]:
import os
import shutil
import cv2
import numpy as np
import threading
import queue
import time
import gc

# ====================================================
# カクつき検知ワーカー（最初のフレームのみ保存）
# ====================================================
def stutter_worker_first_only(frame_queue, output_folder, stop_flag, fps=60, threshold=100, min_time_diff=0.1, stop_no_diff_sec=300):
    min_frame_diff = int(min_time_diff * fps)
    prev_gray = None
    temp_stutter_indices = []
    stutter_frames = []
    last_diff_time = time.time()

    os.makedirs(output_folder, exist_ok=True)

    while True:
        item = frame_queue.get()
        if item is None:
            frame_queue.task_done()
            break

        idx, frame = item
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            non_zero_count = np.count_nonzero(diff)

            if non_zero_count <= threshold:
                temp_stutter_indices.append((idx, frame))
            else:
                if len(temp_stutter_indices) >= min_frame_diff:
                    first_idx, first_frame = temp_stutter_indices[0]
                    stutter_frames.append(first_idx)
                    cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)
                temp_stutter_indices = []
                last_diff_time = time.time()  # 差分ありで更新

        prev_gray = gray

        # 差分無しで指定秒以上なら停止
        if time.time() - last_diff_time > stop_no_diff_sec:
            print("5分以上変化なしのためキャプチャ停止（ワーカー）")
            stop_flag.set()
            break

        # メモリ解放
        del frame, gray
        gc.collect()
        frame_queue.task_done()

    # 残り処理
    if len(temp_stutter_indices) >= min_frame_diff:
        first_idx, first_frame = temp_stutter_indices[0]
        stutter_frames.append(first_idx)
        cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)

    print(f"{len(stutter_frames)} フレームがカクつきとして検出され、最初のフレームのみ保存されました。")

# ====================================================
# キャプチャ＆逐次カクつき検知（メモリ効率版）
# ====================================================
def start_capture_and_detect(device_number, width=720, height=480,
                             capture_fps=60, display_fps=10, threshold=100, 
                             min_time_diff=0.1, max_temp_frames=36000, stop_no_diff_sec=300):
    # 一時フォルダ作成
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    temp_folder = os.path.join(desktop, "temp_frames")
    output_folder = os.path.join(desktop, "stutter_frames")

    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    os.makedirs(temp_folder, exist_ok=True)
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    print(f"{temp_folder} と {output_folder} を作成しました。")

    # キャプチャ初期化
    cap = cv2.VideoCapture(device_number, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Error: キャプチャデバイスを開けませんでした")
        return 0, output_folder, 0

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, capture_fps)

    # 停止フラグ
    stop_flag = threading.Event()

    # カクつき検知用キュー＆スレッド
    frame_queue = queue.Queue(maxsize=5)  # 小さめでメモリ増加防止
    worker = threading.Thread(target=stutter_worker_first_only,
                              args=(frame_queue, output_folder, stop_flag, capture_fps, threshold, min_time_diff, stop_no_diff_sec),
                              daemon=True)
    worker.start()

    frame_count = 0
    display_interval = 1.0 / display_fps
    prev_display_time = time.time()
    start_time = time.time()

    print("映像キャプチャ開始。'q'キーで終了できます。")

    while cap.isOpened():
        if stop_flag.is_set():
            print("停止フラグ検知：メインループ終了")
            break

        ret, frame = cap.read()
        if not ret:
            break

        # 一時保存
        frame_file = os.path.join(temp_folder, f"frame_{frame_count:05d}.png")
        cv2.imwrite(frame_file, frame)

        # キューに追加
        try:
            frame_queue.put_nowait((frame_count, frame.copy()))
        except queue.Full:
            del frame
            gc.collect()

        # 一時フォルダクリア
        if frame_count != 0 and frame_count % max_temp_frames == 0:
            shutil.rmtree(temp_folder)
            os.makedirs(temp_folder, exist_ok=True)
            print(f"{max_temp_frames} 枚保存後、一時フォルダをクリアしました")

        # プレビュー表示
        curr_time = time.time()
        if curr_time - prev_display_time >= display_interval:
            fps_display = 1.0 / (curr_time - prev_display_time)
            prev_display_time = curr_time
            display_frame = frame.copy()
            cv2.putText(display_frame, f"Display FPS: {fps_display:.2f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Preview (FPS limited)", display_frame)
            del display_frame

        frame_count += 1

        # メモリ解放
        del frame
        gc.collect()

        # 'q'キーで終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("ユーザーによる終了")
            break

    # キャプチャ終了
    cap.release()
    cv2.destroyAllWindows()

    # ワーカー終了
    frame_queue.put(None)
    frame_queue.join()

    total_time = time.time() - start_time
    actual_fps = frame_count / total_time if total_time > 0 else 0
    print(f"{frame_count} フレームをキャプチャしました（実際のFPS: {actual_fps:.2f}）")
    return frame_count, output_folder, actual_fps


# ====================================================
# 実行例
# ====================================================
if __name__ == "__main__":
    start_capture_and_detect(device_number=0, capture_fps=60, display_fps=10,
                             threshold=100, min_time_diff=0.1)


C:\Users\HP_PC\Desktop\temp_frames と C:\Users\HP_PC\Desktop\stutter_frames を作成しました。
映像キャプチャ開始。'q'キーで終了できます。
5分以上変化なしのためキャプチャ停止（ワーカー）
停止フラグ検知：メインループ終了
4 フレームがカクつきとして検出され、最初のフレームのみ保存されました。


↓動作する※動作するやつのメモリ消費改善版（再改良版）
確認項目
１．5分以上の差分なしは停止として処理→確認中
２．36000枚（10分）の画像がたまったら画像データを破棄
３．以前の一時ファイルに大量の画像があるといけない（処理しきれてない）→処理に時間が掛かってる

In [1]:
import os
import shutil
import cv2
import numpy as np
import threading
import queue
import time
import gc

# ====================================================
# 🔒 安全で完全なフォルダ削除関数（Windows対応・再試行付き）
# ====================================================
def safe_rmtree(folder_path: str, retry=3, wait=0.5):
    """
    指定フォルダを安全に削除（完全削除・例外保護・再試行あり）

    Args:
        folder_path (str): 削除するフォルダのパス
        retry (int): 失敗時の再試行回数
        wait (float): 再試行前の待機時間（秒）
    """
    if not os.path.exists(folder_path):
        return

    for attempt in range(retry):
        try:
            shutil.rmtree(folder_path)
            break  # 成功
        except PermissionError as e:
            print(f"⚠ PermissionError: {e} → {wait}秒後に再試行 ({attempt+1}/{retry})")
            time.sleep(wait)
        except FileNotFoundError:
            break
        except Exception as e:
            print(f"⚠ 削除中エラー: {e} → 再試行 ({attempt+1}/{retry})")
            time.sleep(wait)
    else:
        print(f"❌ {folder_path} の削除に失敗しました。")

    # 残っている場合は手動削除（完全削除）
    if os.path.exists(folder_path):
        for root, dirs, files in os.walk(folder_path, topdown=False):
            for name in files:
                try:
                    os.remove(os.path.join(root, name))
                except Exception:
                    pass
            for name in dirs:
                try:
                    os.rmdir(os.path.join(root, name))
                except Exception:
                    pass
        try:
            os.rmdir(folder_path)
        except Exception as e:
            print(f"⚠ 最終削除でも残存: {e}")
    else:
        print(f"🗑️ 完全削除済み: {folder_path}")


# ====================================================
# カクつき検知ワーカー（最初のフレームのみ保存）
# ====================================================
def stutter_worker_first_only(frame_queue, output_folder, stop_flag,
                              fps=60, threshold=100, min_time_diff=0.1, stop_no_diff_sec=300):
    min_frame_diff = int(min_time_diff * fps)
    prev_gray = None
    temp_stutter_indices = []
    stutter_frames = []
    last_diff_time = time.time()

    os.makedirs(output_folder, exist_ok=True)

    try:
        while not stop_flag.is_set():
            try:
                item = frame_queue.get(timeout=1)
            except queue.Empty:
                continue

            if item is None:
                frame_queue.task_done()
                break

            idx, frame = item
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if prev_gray is not None:
                diff = cv2.absdiff(gray, prev_gray)
                non_zero_count = np.count_nonzero(diff)

                if non_zero_count <= threshold:
                    temp_stutter_indices.append((idx, frame))

                    # 🔧 静止中メモリ肥大防止：保持上限
                    if len(temp_stutter_indices) > min_frame_diff * 3:
                        temp_stutter_indices.pop(0)

                else:
                    if len(temp_stutter_indices) >= min_frame_diff:
                        first_idx, first_frame = temp_stutter_indices[0]
                        stutter_frames.append(first_idx)
                        cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)
                    temp_stutter_indices = []
                    last_diff_time = time.time()

            prev_gray = gray

            # 差分無しが指定秒続いた場合 → 停止
            if time.time() - last_diff_time > stop_no_diff_sec:
                print("⚠ 5分以上変化なし：キャプチャ停止（ワーカー）")
                stop_flag.set()
                cv2.destroyAllWindows()  # プレビューウィンドウを閉じる
                break

            frame_queue.task_done()
            del frame, gray

            # 🔄 定期メモリ掃除
            if len(temp_stutter_indices) > 0 and (time.time() - last_diff_time) > 5:
                temp_stutter_indices = temp_stutter_indices[-min_frame_diff:]
                gc.collect()

    finally:
        if len(temp_stutter_indices) >= min_frame_diff:
            first_idx, first_frame = temp_stutter_indices[0]
            stutter_frames.append(first_idx)
            cv2.imwrite(os.path.join(output_folder, f"stutter_{first_idx:05d}.png"), first_frame)

        print(f"✅ カクつき検出終了: {len(stutter_frames)} フレーム保存完了")
        gc.collect()

# ====================================================
# キャプチャ＆逐次カクつき検知（完全終了版）
# ====================================================
def start_capture_and_detect(device_number, width=720, height=480,
                             capture_fps=60, display_fps=10, threshold=100,
                             min_time_diff=0.1, max_temp_frames=36000,
                             stop_no_diff_sec=300):

    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    temp_folder = os.path.join(desktop, "temp_frames")
    output_folder = os.path.join(desktop, "stutter_frames")

    # 🔧 フォルダ初期化（安全削除）
    for folder in (temp_folder, output_folder):
        safe_rmtree(folder)
        os.makedirs(folder, exist_ok=True)
    print(f"📁 {temp_folder} と {output_folder} を作成しました。")

    cap = cv2.VideoCapture(device_number, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("❌ Error: キャプチャデバイスを開けませんでした。")
        return 0, output_folder, 0

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, capture_fps)

    stop_flag = threading.Event()
    frame_queue = queue.Queue(maxsize=5)

    worker = threading.Thread(
        target=stutter_worker_first_only,
        args=(frame_queue, output_folder, stop_flag, capture_fps, threshold, min_time_diff, stop_no_diff_sec),
        daemon=True
    )
    worker.start()

    frame_count = 0
    display_interval = 1.0 / display_fps
    prev_display_time = time.time()
    start_time = time.time()

    print("🎥 映像キャプチャ開始。'q'キーで終了できます。")

    try:
        while cap.isOpened():
            if stop_flag.is_set():
                print("🛑 停止フラグ検知：メインループ終了")
                break

            ret, frame = cap.read()
            if not ret:
                print("⚠ フレーム取得失敗、停止します。")
                break

            frame_file = os.path.join(temp_folder, f"frame_{frame_count:05d}.png")
            cv2.imwrite(frame_file, frame)

            try:
                frame_queue.put_nowait((frame_count, frame.copy()))
            except queue.Full:
                pass  # 古いフレーム破棄でメモリ増加防止

            curr_time = time.time()
            if curr_time - prev_display_time >= display_interval:
                fps_display = 1.0 / (curr_time - prev_display_time)
                prev_display_time = curr_time
                display_frame = frame.copy()
                cv2.putText(display_frame, f"Display FPS: {fps_display:.2f}",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow("Preview (FPS limited)", display_frame)
                del display_frame

            frame_count += 1

            # 定期的に一時フォルダ削除
            if frame_count % max_temp_frames == 0:
                safe_rmtree(temp_folder)
                os.makedirs(temp_folder, exist_ok=True)
                print(f"🧹 一時フォルダをクリアしました（{max_temp_frames}フレーム到達）")

            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("🧍 ユーザーによる終了操作")
                stop_flag.set()
                break

            if frame_count % 300 == 0:
                gc.collect()

    finally:
        stop_flag.set()
        frame_queue.put(None)
        frame_queue.join()
        worker.join(timeout=3)
        cap.release()
        cv2.destroyAllWindows()
        gc.collect()
        total_time = time.time() - start_time
        actual_fps = frame_count / total_time if total_time > 0 else 0
        print(f"✅ 完全終了: {frame_count} フレーム（実測FPS: {actual_fps:.2f}）")

    return frame_count, output_folder, actual_fps


# ====================================================
# 実行
# ====================================================
if __name__ == "__main__":
    start_capture_and_detect(
        device_number=0,
        capture_fps=60,
        display_fps=10,
        threshold=100,
        min_time_diff=0.1
    )


🗑️ 完全削除済み: C:\Users\HP_PC\Desktop\temp_frames
🗑️ 完全削除済み: C:\Users\HP_PC\Desktop\stutter_frames
📁 C:\Users\HP_PC\Desktop\temp_frames と C:\Users\HP_PC\Desktop\stutter_frames を作成しました。
🎥 映像キャプチャ開始。'q'キーで終了できます。
⚠ 5分以上変化なし：キャプチャ停止（ワーカー）
🛑 停止フラグ検知：メインループ終了


: 

In [1]:
import cv2, time

# 🎥 内蔵カメラを使用
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # CAP_DSHOWはWindows高速化用（任意）

# 解像度と理論FPS設定（カメラが対応していれば反映される）
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 60)

# 実測開始
count = 0
start = time.time()

print("📸 FPS計測中...（5秒ほどで自動終了）")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠ カメラからフレームを取得できません。")
        break

    count += 1

    # ESCまたは5秒経過で終了
    if (time.time() - start) > 5:
        break

end = time.time()
cap.release()

# --- 実測FPSを計算 ---
elapsed = end - start
actual_fps = count / elapsed if elapsed > 0 else 0

print(f"📈 実測FPS: {actual_fps:.2f}")
print(f"📸 取得フレーム数: {count}, 測定時間: {elapsed:.2f} 秒")


📸 FPS計測中...（5秒ほどで自動終了）
📈 実測FPS: 59.68
📸 取得フレーム数: 299, 測定時間: 5.01 秒
